In [1]:
from haskellian import funcs as F
import pure_cv as vc
import cv2 as cv
import numpy as np
from scoresheet_models import Model, models, draw
from robust_extraction import templates as ts

In [2]:
xs = ts.models['fcde'].cols.points
bw = models['fcde'].boxWidth

In [10]:
0.06/0.42

0.14285714285714285